In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate


In [2]:
train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [3]:
train.columns


Index(['Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'ERM',
       'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST',
       'AION', 'PT', 'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'CWS',
       'CB', 'ODPM', 'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
       'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL'],
      dtype='object')

In [4]:
X_train = train.drop(columns='Disease_Risk')
y_train = train['Disease_Risk']
X_eval  = eval.drop(columns='Disease_Risk')
y_eval = eval['Disease_Risk']


In [5]:
X_train.sample()


,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,LS,MS,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
ID,,,,,,,,,,,,,,,,,,,,,
1503,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def load_and_preprocess_image(image_id, image_folder, target_size=(224, 224)):
    """
    Load and preprocess an image given its ID.
    """
    image_path = os.path.join(image_folder, f'{image_id}.png')
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0 
    return image


In [8]:
image_folder = '../data/training_images'
images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])


In [9]:
eval_image_folder = '../data/eval_images'
eval_images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_eval.index])


In [31]:
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)


row_input = Input(shape=(X_train.shape[1],))
y = Dense(64, activation='relu')(row_input)

combined = concatenate([x, y])

z = Dense(12, activation='relu')(combined)
z = Dense(64, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z) 


model = Model(inputs=[image_input, row_input], outputs=z)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_9 (Conv2D)           (None, 222, 222, 32)         896       ['input_21[0][0]']            
                                                                                                  
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)         0         ['conv2d_9[0][0]']            
 g2D)                                                                                             
                                                                                                  
 input_22 (InputLayer)       [(None, 45)]                 0         []                      

In [33]:
model.fit(
    [images, X_train], y_train,
    validation_data=([eval_images, X_eval], y_eval),
    epochs=10,
    batch_size=32
)


Epoch 1/10
60/60 [==============================] - 17s 268ms/step - loss: 0.5259 - accuracy: 0.8036 - val_loss: 0.5266 - val_accuracy: 0.7906
Epoch 2/10
60/60 [==============================] - 17s 288ms/step - loss: 0.4263 - accuracy: 0.8203 - val_loss: 0.5802 - val_accuracy: 0.7906
Epoch 3/10
60/60 [==============================] - 16s 259ms/step - loss: 0.3637 - accuracy: 0.8411 - val_loss: 0.4132 - val_accuracy: 0.7906
Epoch 4/10
60/60 [==============================] - 25s 412ms/step - loss: 0.2238 - accuracy: 0.9193 - val_loss: 0.1488 - val_accuracy: 0.9828
Epoch 5/10
60/60 [==============================] - 17s 272ms/step - loss: 0.0954 - accuracy: 0.9984 - val_loss: 0.0788 - val_accuracy: 0.9937
Epoch 6/10
60/60 [==============================] - 18s 298ms/step - loss: 0.0497 - accuracy: 1.0000 - val_loss: 0.0432 - val_accuracy: 0.9937
Epoch 7/10
60/60 [==============================] - 16s 264ms/step - loss: 0.0258 - accuracy: 1.0000 - val_loss: 0.0253 - val_accuracy: 0.9953

In [34]:
model.evaluate([eval_images, X_eval], y_eval)


20/20 [==============================] - 1s 68ms/step - loss: 0.0164 - accuracy: 0.9922


[0.016367468982934952, 0.9921875]